In [1]:
import d4rl
import gym
import numpy as np

/home/users/adarsh/anaconda3/envs/diffuser/lib/python3.8/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
pybullet build time: May 20 2022 19:44:17


In [2]:
env = gym.make('maze2d-umaze-v1')
dataset = env.get_dataset()
np.where(dataset['timeouts']==1)

load datafile: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 29.71it/s]


(array([    91,    144,    230, ..., 999592, 999702, 999868]),)

In [8]:
env._max_episode_steps

300

In [18]:
env.state_vector()

array([0.00043799, 0.00079652, 0.04379873, 0.07965174])

In [17]:
env.unwrapped.max_episode_steps = env._max_episode_steps
env.unwrapped.max_episode_steps

300

In [ ]:
{k:v[85:110] for k,v in dataset.items()}

In [3]:
{k:v[0:10] for k,v in dataset.items()}

{'actions': array([[-0.56856084, -0.27724722],
        [-1.        ,  0.6666507 ],
        [-1.        ,  1.        ],
        [-0.57507885,  1.        ],
        [ 0.19998227,  0.3388329 ],
        [-0.85686654,  1.        ],
        [-0.70776117,  1.        ],
        [ 0.2815406 ,  1.        ],
        [ 0.3058027 ,  1.        ],
        [-0.17358942,  1.        ]], dtype=float32),
 'infos/goal': array([[3.036284, 3.022871],
        [3.036284, 3.022871],
        [3.036284, 3.022871],
        [3.036284, 3.022871],
        [3.036284, 3.022871],
        [3.036284, 3.022871],
        [3.036284, 3.022871],
        [3.036284, 3.022871],
        [3.036284, 3.022871],
        [3.036284, 3.022871]], dtype=float32),
 'infos/qpos': array([[1.0856489, 1.9745734],
        [1.0843927, 1.97413  ],
        [1.0807577, 1.9752754],
        [1.0747498, 1.9787997],
        [1.0673866, 1.9846972],
        [1.0605173, 1.9913877],
        [1.0516236, 2.000444 ],
        [1.0410653, 2.0118601],
        [1.

In [4]:
n=15
print(dataset.keys())
print(dataset['actions'][n])
print(dataset['observations'][n])
print(dataset['infos/qpos'][n])
print(dataset['infos/qvel'][n])

dict_keys(['actions', 'infos/goal', 'infos/qpos', 'infos/qvel', 'observations', 'rewards', 'terminals', 'timeouts'])
[-0.6284441   0.09111674]
[ 0.97895855  2.180523   -0.42206582  2.7247157 ]
[0.97895855 2.180523  ]
[-0.42206582  2.7247157 ]


In [5]:
np.where(dataset['timeouts']==1)[0]

array([    91,    144,    230, ..., 999592, 999702, 999868])

In [ ]:
{k:v[0:93] for k,v in dataset_naive.items()}

In [6]:
import copy
from math import atan2, sin, cos, radians
dataset_naive = copy.deepcopy(dataset)
    

def calc_action(start, goal):
    delta_x = goal[0] - start[0]
    delta_y = goal[1] - start[1]
    theta_radians = atan2(delta_y, delta_x)
    a_x = cos(theta_radians)
    a_y = sin(theta_radians)
    return [a_x,a_y]
    

start = 0
for t in np.where(dataset['timeouts']==1)[0]:
    dataset_naive['actions'][start:t+1] = calc_action(dataset['infos/qpos'][start], dataset['infos/goal'][t])
    start = t+1
    
    

In [7]:
dataset_naive['actions']

array([[ 0.88085616,  0.473384  ],
       [ 0.88085616,  0.473384  ],
       [ 0.88085616,  0.473384  ],
       ...,
       [ 0.09495355, -0.12691994],
       [ 1.        , -0.04909204],
       [ 1.        , -1.        ]], dtype=float32)